In [ ]:
## IEX EDA starts here

In [1]:
import os
import requests
import pandas as pd

In [2]:
os.environ.get('IEX_SANDBOX_TOKEN')

'Tsk_2f36139a7da4433bbc4882148386ff32'

In [6]:
base_url = 'https://cloud.iexapis.com/v1'
sandbox_url = 'https://sandbox.iexapis.com/stable'

token = os.environ.get('IEX_TOKEN')
sandbox_token = os.environ.get('IEX_SANDBOX_TOKEN')
params = {'token': token}
sandbox_params = {'token': sandbox_token}


In [36]:
# endpoint = '/stock/aapl/chart'
# endpoint = '/stock/aapl/balance-sheet'
endpoint = '/time-series'
resp = requests.get(sandbox_url+endpoint, params = sandbox_params)

In [37]:
resp.raise_for_status()

In [38]:
resp.json()

[{'inv_pk': 8,
  'id': 'XF',
  'description': 'o a i dri,tpi,  satoSt5srxce Pherroan2laf',
  'schema': {'additionalProperties': True,
   'properties': {'rate': {'type': 'number'},
    'symbol': {'type': 'string'},
    'timestamp': {'type': 'number'}},
   'required': ['aemsittmp', 'taer'],
   'type': 'object'},
  'weight': 104986,
  'created': '2020-05-05T16:58:52.000Z',
  'updated': '2020-05-05T16:58:52.000Z',
  'version': 1,
  'weightKey': 'XF'},
 {'inv_pk': 73,
  'id': 'HSCAOTS_C_VAIEPDUTHAGE_NDRR',
  'description': 'iAcvdiend adeddnsv',
  'schema': {'additionalProperties': True,
   'properties': {'symbol': {'type': 'string'}},
   'required': ['ombsly'],
   'type': 'object'},
  'weight': 0,
  'created': '2019-11-25T14:26:55.000Z',
  'updated': '2019-11-25T14:26:55.000Z',
  'version': 1,
  'weightKey': 'ET_ONCPRSG_AEVCSDRDAAHHT_IU'},
 {'inv_pk': 2054,
  'id': 'AEUMRDNUF_LSDHO_LUT',
  'description': 'aetn dFsHloruudlu M',
  'schema': {'additionalProperties': False,
   'properties': {'a

In [29]:
df = pd.DataFrame(resp.json())

In [30]:
df.head()

,symbol,balancesheet
0,AAPL,"{'accountsPayable': 41958271854, 'capitalSurpl..."


In [16]:
# If you plan to use this same endpoint a few times, it makes sense to create a function to avoid repeating code. Here is an example:

In [17]:
def historical_data(_symbol, _range=None, _date=None):
    endpoint = f'{base_url}/stock/{_symbol}/chart'
    if _range:
        endpoint += f'/{_range}'
    elif _date:
        endpoint += f'/date/{_date}'
    
    resp = requests.get(endpoint, params=params)
    resp.raise_for_status()
    return pd.DataFrame(resp.json())

In [26]:
#appl_3m_df = historical_data('AAPL', _range='1m')

In [23]:
#endpoint = '/stock/market/today-earnings'
#resp = requests.get(sandbox_url+endpoint, params=sandbox_params)

In [25]:
#resp.raise_for_status()